# Strategy Description: Bollinger Band Mean Reversion (CAD)

### 1. Executive Summary
This strategy employs a classic **Mean Reversion** approach applied to the Canadian Dollar (CAD).

The core concept relies on **Bollinger Bands** to identify statistically extreme price levels. By defining overbought and oversold conditions as deviations of 2 standard deviations ($\sigma$) from the moving average, the strategy seeks to capitalize on the price's tendency to revert to its central tendency (the mean) after significant extensions.

### 2. Investment Universe & Specifications
* **Asset:** Canadian Dollar (CAD)
    * Point Value: 100,000 (Standard Lot)
    * Transaction Cost: $5.0 per trade
* **Data Frequency:** 5-minute bars (Intraday)

### 3. Methodology

#### A. Indicator Construction (Bollinger Bands)
The strategy utilizes standard Bollinger Bands to measure volatility and relative price levels dynamically.
* **Central Tendency (Basis):** Simple Moving Average (SMA) of the Close price.
    * Window: 20 bars.
* **Volatility Metric:** Rolling Standard Deviation ($\sigma$) of the Close price (same window).

#### B. Dynamic Thresholds -> Advantages of this approach
[Rationale] Unlike fixed price thresholds, Bollinger Bands automatically adjust to market volatility.
* **Band Expansion:** During high volatility, the bands widen, requiring a larger price move to trigger a signal. This inherently filters out noise during turbulent periods.
* **Band Contraction:** During low volatility, the bands narrow, allowing the strategy to capture smaller revertive moves when the market is quiet.
* **Upper Band:** $MA + (2.0 \times \sigma)$
* **Lower Band:** $MA - (2.0 \times \sigma)$

### 4. Trading Logic

The strategy operates on a "Touch" basis for entries and a "Mean Return" basis for exits.

* **Entry Signals (Counter-Trend):**
    * **Short Signal:** If Price $\ge$ Upper Band ($+2\sigma$) $\rightarrow$ **Sell CAD**.
        * *Logic:* Price is statistically overextended to the upside (Overbought).
    * **Long Signal:** If Price $\le$ Lower Band ($-2\sigma$) $\rightarrow$ **Buy CAD**.
        * *Logic:* Price is statistically overextended to the downside (Oversold).

* **Exit Signals (Mean Reversion):**
    * Positions are closed when the price touches or crosses the **Central Moving Average (MA)**.
    * *Logic:* The statistical anomaly has normalized.

* **Constraints:**
    * Max Position: 1 Unit Long or 1 Unit Short.
    * **Out-of-Sample (OOS):** Trading is strictly disabled during OOS periods.

### 5. Risk Management
* **Volatility Adaptation:** The use of standard deviation ensures that entry criteria become stricter during turbulent market phases, naturally reducing trade frequency during unpredictable spikes.
* **Conservative Exit:** Exiting at the Mean (rather than waiting for the opposite band) secures profits earlier and reduces the duration of exposure to market risk.

In [2]:
import pandas as pd
import numpy as np
import warnings
import os

warnings.filterwarnings('ignore')

# ==========================================
# 1. Configuration (Submission Ready)
# ==========================================
class Config:
    # Quarters for submission
    QUARTERS = ['2023_Q1', '2023_Q3', '2023_Q4', '2024_Q2', '2024_Q4', '2025_Q1', '2025_Q2']
    
    # Target Asset (CAD)
    ASSETS = ['CAD'] 
    
    # Contract Specs (Group 2: CAD)
    # Transaction Cost $10, Point Value $100,000
    SPECS = {
        'CAD': {'point_val': 100000.0, 'cost': 10.0}
    }

    # === Trading Hours Rules (Group 2 Strict) ===
    # Break: 17:00 - 18:00
    # Mandatory Exit: 16:50 (10 mins before break)
    # Restart Trading: 18:10 (10 mins after break)
    EXIT_TIME = pd.to_datetime("16:50").time()
    RESTART_TIME = pd.to_datetime("18:10").time()

    # === Strategy Parameters (Optimized for Quarterly Reset) ===
    # 5-minute data frequency
    # User requested approx 8 hours window.
    # 8 hours * 12 bars/hour = 96 bars
    
    BB_WINDOW = 96     # 8 Hours lookback (Fast start for quarterly data)
    BB_SIGMA = 2.0     # Standard deviation threshold
    
    # Annualization factor for Sharpe scaling
    ANNUALIZATION = 252

def mySR(daily_pnl, scale=252):
    """Calculates Annualized Sharpe Ratio based on Daily PnL"""
    if np.nanstd(daily_pnl) == 0: return 0
    return np.sqrt(scale) * np.nanmean(daily_pnl) / np.nanstd(daily_pnl)

def find_data_path(quarter, prefix='data2'):
    """Helper to find data in various relative paths"""
    filename = f'{prefix}_{quarter}.parquet'
    potential_paths = [
        f'data/{filename}',          # Same folder
        f'../data/{filename}',       # Up one level
        f'../../data/{filename}',    # Up two levels
        f'../../../data/{filename}', # Up three levels
        filename                     # Current folder
    ]
    for path in potential_paths:
        if os.path.exists(path): return path
    return None

# ==========================================
# 2. Strategy Engine (Per-Quarter Logic)
# ==========================================
print("\n--- Calculating Group 2 (CAD Bollinger Bands) Stats ---")
results = []

for quarter in Config.QUARTERS:
    try:
        # --- 1. Data Loading ---
        file_path = find_data_path(quarter, prefix='data2')
        if file_path is None:
            raise FileNotFoundError(f"File not found for {quarter}")

        df = pd.read_parquet(file_path)
        df.set_index('datetime', inplace=True)

        # --- 2. Data Cleaning & Time Filtering ---
        # Remove break data (17:00-18:00) strictly
        df.loc[df.between_time("17:00", "18:00").index] = np.nan
        
        # --- 3. Indicators Calculation ---
        price = df['CAD']
        
        # Calculate Bollinger Bands
        # min_periods=BB_WINDOW ensures we don't trade on unstable initial data
        ma = price.rolling(window=Config.BB_WINDOW, min_periods=Config.BB_WINDOW).mean()
        std = price.rolling(window=Config.BB_WINDOW, min_periods=Config.BB_WINDOW).std()
        
        upper = ma + (Config.BB_SIGMA * std)
        lower = ma - (Config.BB_SIGMA * std)
        
        # --- 4. Iterative Execution Loop ---
        times = df.index.time
        price_vals = price.values
        upper_vals = upper.values
        lower_vals = lower.values
        ma_vals = ma.values
        
        pos_cad = np.zeros(len(df))
        curr_pos = 0
        
        for i in range(len(df)):
            t = times[i]
            p = price_vals[i]
            
            # --- Rule: Mandatory Exit at 16:50 ---
            if t >= Config.EXIT_TIME and t < Config.RESTART_TIME:
                curr_pos = 0
                pos_cad[i] = 0
                continue
                
            # Skip if indicators are NaN
            if np.isnan(upper_vals[i]) or np.isnan(lower_vals[i]):
                pos_cad[i] = curr_pos
                continue

            u = upper_vals[i]
            l = lower_vals[i]
            m = ma_vals[i]

            # --- Strategy Logic: Mean Reversion ---
            if curr_pos == 0:
                if p > u:
                    # Price above Upper Band -> Short
                    curr_pos = -1
                elif p < l:
                    # Price below Lower Band -> Long
                    curr_pos = 1
            
            elif curr_pos == 1: # Long
                # Exit if price recovers to MA (or goes higher)
                if p >= m:
                    curr_pos = 0
                    
            elif curr_pos == -1: # Short
                # Exit if price drops to MA (or goes lower)
                if p <= m:
                    curr_pos = 0
            
            pos_cad[i] = curr_pos

        # --- 5. PnL Calculation ---
        # Shift positions by 1
        pos_held = pd.Series(pos_cad, index=df.index).shift(1).fillna(0)
        
        # SAFETY CHECK: Ensure positions are 0 during forbidden times
        mask_forbidden = (pd.Series(df.index, index=df.index).dt.time >= Config.EXIT_TIME) & \
                         (pd.Series(df.index, index=df.index).dt.time < Config.RESTART_TIME)
        pos_held[mask_forbidden] = 0

        # Calculate PnL (Gross)
        pnl_gross = pos_held * df['CAD'].diff() * Config.SPECS['CAD']['point_val']
        
        # Calculate Costs
        trades = pos_held.diff().abs().fillna(0)
        total_cost = trades * Config.SPECS['CAD']['cost']
        
        pnl_net = pnl_gross - total_cost
        
        # --- 6. Aggregation & Metrics ---
        daily_net = pnl_net.resample('D').sum()
        # Remove non-trading days
        daily_net = daily_net[daily_net != 0] 
        
        net_sr = mySR(daily_net, 252)
        net_cum = daily_net.sum()
        
        # === PROFESSOR'S STATISTIC FORMULA ===
        abs_pnl_scaled = abs(net_cum) / 1000.0
        log_term = np.log(abs_pnl_scaled) if abs_pnl_scaled > 0 else 0
        stat_val = (net_sr - 0.5) * max(0, log_term)

        results.append({
            'Quarter': quarter,
            'Net SR': round(net_sr, 2),
            'Net PnL': round(net_cum, 2),
            'Trades/Day': round(trades.resample('D').sum().mean(), 2),
            'Stat': round(stat_val, 2)
        })

    except Exception as e:
        print(f"Skipping {quarter}: {e}")

# ==========================================
# 3. Final Report (Console Only)
# ==========================================
df_res = pd.DataFrame(results)
print("\n=== Group 2 (CAD) Results ===")
print(df_res.to_markdown(index=False))

if not df_res.empty:
    print(f"\nTotal Stat: {df_res['Stat'].sum():.2f}")


--- Calculating Group 2 (CAD Bollinger Bands) Stats ---

=== Group 2 (CAD) Results ===
| Quarter   |   Net SR |   Net PnL |   Trades/Day |   Stat |
|:----------|---------:|----------:|-------------:|-------:|
| 2023_Q1   |    -1.15 |  -1267.51 |         3.73 |  -0.39 |
| 2023_Q3   |    -3.08 |  -2277.19 |         3.38 |  -2.94 |
| 2023_Q4   |    -1.83 |  -1361.67 |         3.6  |  -0.72 |
| 2024_Q2   |    -2.61 |  -1629.47 |         3.58 |  -1.52 |
| 2024_Q4   |    -5.19 |  -3383.67 |         3.3  |  -6.94 |
| 2025_Q1   |    -2.75 |  -2525.57 |         3.27 |  -3.01 |
| 2025_Q2   |    -5.65 |  -4077.17 |         3.43 |  -8.64 |

Total Stat: -24.16
